# FULL SOLUTION - LOOPING THROUGH ALL PLANTS IN GREECE -  Loading ALL WASTEWATER PLANTS - SCRAPING FROM YPEN 

# WASTEWATER TREATMENT PLANTS - SENTINELEGXOS -- SWIM WATER EVALUATION TOOL

### STEP 1: REQUEST BUILDER -- BUILD THE API 

Request Builder is an online graphical interface to the Sentinel Hub API-s. This tool makes it easier to create and debug API requests, and supports the export of the resulting code in various programming languages. In this tutorial, we will create a Process API request for downloading raster images of the burnt area from the location of the wildfire in Greece that we have already examined in the Browser. Just like a Process API request in code, a request created with the Requests Builder consists of 5 main parts:

    Data Collection
    Time Range
    Area of Interest
    Output
    Evalscript

Screenshot of Requests Builder for the ΑΙΤΩΛΙΚΟ are in AITOLOAKARNANIA GREECE

These can be set individually in the interface. Use the following settings for the Alexandropouli Wildfire example here:

    Data Collection: sentinel-2 l2a
    Time Range: From 2025-09-23 to 2025-09-23
    Area of Interest: [
  21.336075198,
  38.445840322,
  21.392800692,
  38.399042588
]

Click Parse to parse the area of interest - it should be displayed in the map window and zoom in to the rectangle of interest.

    Evalscript: use the evalscript from the previous Copernicus Browser example:



/*
Name:    Sentinel-2 Water Quality (Se2WaQ) 
Version: 1.0
Date:    2020-01-31

Author:      Nuno Sidónio Andrade Pereira
Affiliation: Polytechnic Institute of Beja, Portugal
License:     Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)

Refs.: [1]M. Potes et al., “Use of Sentinel 2 – MSI for water quality 
          monitoring at Alqueva reservoir, Portugal,” Proc. Int. Assoc. 
          Hydrol. Sci., vol. 380, pp. 73–79, Dec. 2018.

       [2]K. Toming, T. Kutser, A. Laas, M. Sepp, B. Paavel, and T. Nõges,
          “First Experiences in Mapping Lake Water Quality Parameters with
          Sentinel-2 MSI Imagery,” Remote Sens., vol. 8, no. 8, p. 640, 
          Aug. 2016.
*/

// user defined FLAGs

var FLAGparam = 5;
var FLAGbackGround = 2;

// Water-land contrast index (to define the background)

var NDWI = index(B03, B08); 

// Background indexes                           

var Black = [0];                                       // FLAGbackGround = 0

var NDVI = index(B08, B04);                            // FLAGbackGround = 1

var TrueColor = [B04*2.5, B03*2.5, B02*2.5];           // FLAGbackGround = 2


// Empirical models

var Chl_a = 4.26 * Math.pow(B03/B01, 3.94);            // FLAGparam = 0; S2-L2A; [1] Unit: mg/m3;        

var Cya = 115530.31 * Math.pow(B03 * B04 / B02, 2.38); // FLAGparam = 1; S2-L2A; [1] Unit: 10^3 cell/ml; 

var Turb = 8.93 * (B03/B01) - 6.39;                    // FLAGparam = 2; S2-L2A; [1] Unit: NTU;          

var CDOM = 537 * Math.exp(-2.93*B03/B04);              // FLAGparam = 3; S2-L1C; [2] Unit: mg/l;         

var DOC = 432 * Math.exp(-2.24*B03/B04);               // FLAGparam = 4; S2-L1C; [2] Unit: mg/l;         

var Color = 25366 * Math.exp(-4.53*B03/B04);           // FLAGparam = 5; S2-L1C; [2] Unit: mg.Pt/l;      


// Numerical values for the scales of parameters

var scaleChl_a = [0, 6, 12, 20, 30, 50];
var scaleCya   = [0, 10, 20, 40, 50, 100];
var scaleTurb  = [0, 4, 8, 12, 16, 20];
var scaleCDOM  = [0, 1, 2, 3, 4, 5];
var scaleDOC   = [0, 5, 10, 20, 30, 40];
var scaleColor = [0, 10, 20, 30, 40, 50];

// Colors for the scales

var s = 255;
var colorScale = 
  [
   [73/s, 111/s, 242/s],
   [130/s, 211/s, 95/s],
   [254/s, 253/s, 5/s],
   [253/s, 0/s, 4/s],
   [142/s, 32/s, 38/s],
   [217/s, 124/s, 245/s]
  ];

// Image generation

if (NDWI<0) {
  if ( FLAGbackGround == 0 ) {
    return Black;
  } else if ( FLAGbackGround == 1 ) {
    return [0, .5*(NDVI+1), 0];
  } else if ( FLAGbackGround == 2 ) {
    return TrueColor;
  }
} else {
  switch ( FLAGparam ) {
    case 0:
     return colorBlend(Chl_a, scaleChl_a, colorScale);
     break;
    case 1:
      return colorBlend(Cya, scaleCya, colorScale);
      break;
    case 2:
      return colorBlend(Turb, scaleTurb, colorScale);
      break;
    case 3:
      return colorBlend(CDOM, scaleCDOM, colorScale);
      break;
    case 4:
      return colorBlend(DOC, scaleDOC, colorScale);
      break;
    case 5:
      return colorBlend(Color, scaleColor, colorScale);
      break;
    default:
      return TrueColor;
  }
}




Finally, you must set the language of the Request Preview to python-requests in the dropdown menu. This will create request code you can later use in the Jupyter Notebook as well.

If you now click on Send request, you will be prompted to save the request and download the response. The response is a .tar file containing default.tif and burnMask.tif. To download raster datasets of the burnt area for a time series, you only need to change the Time Range parameter and repeat running the request. Possible dates can be the following:

    03.08.2023 - 04.08.2023
    23.08.2023 - 24.08.2023
    28.08.2023 - 29.08.2023
    02.09.2023 - 03.08.2023
    12.08.2023 - 13.08.2023

The result is a series of TIFF files,       which you can process locally in GIS software to calculate quantitative results. But more importantly, you can copy the code from the Request Preview window and use it in a Jupyter Notebook of the same test case.
Step 3: Jupyter notebooks
Importing necessary libraries

To run this example, you do not need any additional GIS-specific libraries. You can of course improve the workflow with additional libraries, but for someone new to earth observation coding, this basic notebook could be a good place to start.

## STEP 2 AUTOMATE THE RETRIEVAL WITH PYTHON

In [ ]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
from sentinelhub import SHConfig
import matplotlib.pyplot as plt
import numpy as np
import tarfile
import getpass


### Credentials

You can obtain credentials for the Sentinel Hub services (`client_id` & `client_secret`) by navigating to your [Dashboard](https://shapps.dataspace.copernicus.eu/dashboard/#/). In the User Settings, you can create a new OAuth Client to generate these credentials. More detailed instructions can be found on the  corresponding [documentation page](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html).

Once you run the next cell, you will be prompted to enter the `client_id` and `client_secret`.

In [ ]:
# Your client credentials

config = SHConfig()
config.sh_client_id = getpass.getpass("Enter your SentinelHub ID")
config.sh_client_secret = getpass.getpass("Enter your SentinelHub client secret")
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
config.save("cdse")

If you have the credentials, you will need a session token to make requests. This token is generated in the following function.

In [ ]:
def getauth_token():
    # Create a session
    client = BackendApplicationClient(client_id=config.sh_client_id)
    oauth = OAuth2Session(client=client)
    # Get token for the session
    token = oauth.fetch_token(
        token_url="https://identity.cloudferro.com/auth/realms/CDSE/protocol/openid-connect/token",
        client_id=config.sh_client_id,
        client_secret=config.sh_client_secret,
    )
    # All requests using this session will have an access token automatically added
    print ("oauth=",oauth)
    return oauth
   

## Defining time slots 

Using the [Requests Builder](https://shapps.dataspace.copernicus.eu/requests-builder/) and the Browser, we can see all the images acquisitions that match our criteria. In the following cell, we enter these time slots to create a time series and understand the extent of the damage caused.

In [ ]:
time_slots = [
    ("2022-04-01", "2022-04-07"),
    ("2022-06-01", "2022-06-07"),
    ("2022-08-01", "2022-08-07"),
    ("2022-10-01", "2022-10-07"),
    ("2023-04-01", "2023-04-07"),
    ("2023-06-01", "2023-06-07"),
    ("2023-08-01", "2023-08-07"),
    ("2025-10-01", "2023-10-07"),
    ("2025-04-01", "2025-04-07"),
    ("2025-06-01", "2025-06-07"),
    ("2025-08-01", "2025-08-07")
]
print("Time Slots:\n")
for slot in time_slots:
    print(slot[0])

Next, we can enter the evalscript for wastewater map that we used earlier in the Browser and Requests Builder - feel free to copy it from the Browser window.

In [ ]:
evalscript = """
/*
Name:    Sentinel-2 Water Quality (Se2WaQ) 
Version: 1.0
Date:    2020-01-31

Author:      Nuno Sidónio Andrade Pereira
Affiliation: Polytechnic Institute of Beja, Portugal
License:     Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)

Refs.: [1]M. Potes et al., “Use of Sentinel 2 – MSI for water quality 
          monitoring at Alqueva reservoir, Portugal,” Proc. Int. Assoc. 
          Hydrol. Sci., vol. 380, pp. 73–79, Dec. 2018.

       [2]K. Toming, T. Kutser, A. Laas, M. Sepp, B. Paavel, and T. Nõges,
          “First Experiences in Mapping Lake Water Quality Parameters with
          Sentinel-2 MSI Imagery,” Remote Sens., vol. 8, no. 8, p. 640, 
          Aug. 2016.
*/

// user defined FLAGs

var FLAGparam = 5;
var FLAGbackGround = 2;

// Water-land contrast index (to define the background)

var NDWI = index(B03, B08); 

// Background indexes                           

var Black = [0];                                       // FLAGbackGround = 0

var NDVI = index(B08, B04);                            // FLAGbackGround = 1

var TrueColor = [B04*2.5, B03*2.5, B02*2.5];           // FLAGbackGround = 2


// Empirical models

var Chl_a = 4.26 * Math.pow(B03/B01, 3.94);            // FLAGparam = 0; S2-L2A; [1] Unit: mg/m3;        

var Cya = 115530.31 * Math.pow(B03 * B04 / B02, 2.38); // FLAGparam = 1; S2-L2A; [1] Unit: 10^3 cell/ml; 

var Turb = 8.93 * (B03/B01) - 6.39;                    // FLAGparam = 2; S2-L2A; [1] Unit: NTU;          

var CDOM = 537 * Math.exp(-2.93*B03/B04);              // FLAGparam = 3; S2-L1C; [2] Unit: mg/l;         

var DOC = 432 * Math.exp(-2.24*B03/B04);               // FLAGparam = 4; S2-L1C; [2] Unit: mg/l;         

var Color = 25366 * Math.exp(-4.53*B03/B04);           // FLAGparam = 5; S2-L1C; [2] Unit: mg.Pt/l;      


// Numerical values for the scales of parameters

var scaleChl_a = [0, 6, 12, 20, 30, 50];
var scaleCya   = [0, 10, 20, 40, 50, 100];
var scaleTurb  = [0, 4, 8, 12, 16, 20];
var scaleCDOM  = [0, 1, 2, 3, 4, 5];
var scaleDOC   = [0, 5, 10, 20, 30, 40];
var scaleColor = [0, 10, 20, 30, 40, 50];

// Colors for the scales

var s = 255;
var colorScale = 
  [
   [73/s, 111/s, 242/s],
   [130/s, 211/s, 95/s],
   [254/s, 253/s, 5/s],
   [253/s, 0/s, 4/s],
   [142/s, 32/s, 38/s],
   [217/s, 124/s, 245/s]
  ];

// Image generation

if (NDWI<0) {
  if ( FLAGbackGround == 0 ) {
    return Black;
  } else if ( FLAGbackGround == 1 ) {
    return [0, .5*(NDVI+1), 0];
  } else if ( FLAGbackGround == 2 ) {
    return TrueColor;
  }
} else {
  switch ( FLAGparam ) {
    case 0:
     return colorBlend(Chl_a, scaleChl_a, colorScale);
     break;
    case 1:
      return colorBlend(Cya, scaleCya, colorScale);
      break;
    case 2:
      return colorBlend(Turb, scaleTurb, colorScale);
      break;
    case 3:
      return colorBlend(CDOM, scaleCDOM, colorScale);
      break;
    case 4:
      return colorBlend(DOC, scaleDOC, colorScale);
      break;
    case 5:
      return colorBlend(Color, scaleColor, colorScale);
      break;
    default:
      return TrueColor;
  }
}
"""
#print ("evalscript=",evalscript)
print ("evalscript Read OK")

Here, we have defined a function that contains the request code made in the Requests Builder and references the evalscript defined above. Since we are requesting multiple files in the output, we can expect a compressed .tar file containing the files.

In [ ]:
def get_request(slot):
    request = {
        "input": {
            "bounds": {
                "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
                "bbox": [  21.336075198,  38.445840322,  21.392800692,  38.399042588],
            },
            "data": [
                {
                    "dataFilter": {
                        "timeRange": {
                            "from": slot[0] + "T00:00:00Z",
                            "to": slot[1] + "T00:00:00Z",
                        }
                    },
                    "type": "sentinel-2-l2a",
                }
            ],
        },
        "output": {
            "width": 1247.7098306086236,
            "height": 1031.9962449583074,
            "responses": [
                {
                    "identifier": "default",
                    "format": {"type": "image/tiff"},
                }
            ],
        },
        "evalscript": evalscript,
    }

    url = "https://sh.dataspace.copernicus.eu/api/v1/process"
    response = oauth.post(url, json=request, headers={"Accept": "application/tar"})
    print(response.status_code)

    if response.status_code in (200,):
        with open(f"tarfile_wildfire_{slot[0]}.tar", "wb") as tarfile:
            tarfile.write(response.content)

    return response

Now, we are ready to make the request for each of the previously defined time slots. We first get the token created with the OAuth client before making the request and loop through the request for all time slots. This cell might take 10-20 seconds as this is where the requests are actually performed. If it runs successfully, we should see that the compressed .tar files are created in the local folder. 

In [ ]:
oauth = getauth_token()
responses = [get_request(slot) for slot in time_slots]

To further display and analyse the images further, we need to extract the files. This cell does exactly that and we can see the names of the files in the extracted folder as a result. 

In [ ]:
for slot in time_slots:
    # open file
    file = tarfile.open(f"tarfile_wildfire_{slot[0]}.tar")

    # print file names
    print(file.getnames())

    # extract files
    file.extractall(f"./wildfire_{slot[0]}")

    # close file
    file.close()

In [ ]:
from IPython.display import FileLink

# Replace 'your_file.tar' with the actual filename
FileLink('wildfire_2023-08-03.tar')

Now, we create a series of plots to display the images we have requested. If we look at them together, we can see how the damaged area has increased over time.


ncols = 3
nrows = 2
aspect_ratio = 1000 / 1000
subplot_kw = {"xticks": [], "yticks": [], "frame_on": False}

fig, axs = plt.subplots(
    ncols=ncols,
    nrows=nrows,
    figsize=(5 * ncols * aspect_ratio, 5 * nrows),
    subplot_kw=subplot_kw,
)

for idx, slot in enumerate(time_slots):
    img = plt.imread(f"wildfire_{slot[0]}/default.tif")
    ax = axs[idx // ncols][idx % ncols]
    ax.imshow(img)
    ax.set_title(f"{slot[0]}  -  {slot[1]}", fontsize=10)

plt.tight_layout()

We can visualise the increasing damaged area by plotting these burnt pixels on a simple line chart as shown below.

xlabels = [slot[0] for slot in time_slots]
x = range(len(time_slots))
plt.plot(range(len(time_slots)), burnt_area_arr)
plt.title("Time Series of area burnt in the wildfires.")
plt.xticks(np.arange(0, 5, step=1), xlabels, rotation=30, ha="center")
plt.xlabel("Time slots")
plt.ylabel("Area burnt (in $km^2$)")
plt.show()

This was a brief guide on how to go from viewing satellite imagery in the Copernicus Browser using [Custom Scripts](https://custom-scripts.sentinel-hub.com/custom-scripts/) and requests from the [Requests Builder](https://shapps.dataspace.copernicus.eu/requests-builder/) to preparing a Jupyter Notebook to begin your analysis. From here, you can analyse the pixels, derive statistics and create a workflow that is suitable for your problem. The next step could be to familiarize yourself with various other Sentinel Hub API-s, using [this notebook example](https://github.com/eu-cdse/notebook-samples/blob/main/sentinelhub/introduction_to_SH_APIs.ipynb). You can find more information about the different APIs and various examples in the [documentation](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub.html). Check out the [Custom Scripts Repository](https://custom-scripts.sentinel-hub.com/custom-scripts/) for more examples of evalscripts for a wide range of applications. 

To enhance your Jupyter Notebook for downloading Sentinel-2 SE2WQA images for each wastewater treatment plant in Greece, you need to:

Read the JSON file containing the wastewater treatment plants.
Convert each plant's coordinates to the Greek Grid CRS (EPSG:2100).
Create a 4 km square around each plant.
Convert the square back to WGS84 for use with the Sentinel Hub API.
Loop through each plant and time slot to download the Sentinel-2 images.

Here's how you can integrate this into your existing Jupyter Notebook:

In [ ]:
!pip install pyproj shapely requests numpy matplotlib


In [11]:
time_slots = [
    ("2022-04-01", "2022-04-07"),
    ("2022-06-01", "2022-06-07"),
    ("2022-07-01", "2022-07-07"),
    ("2022-08-01", "2022-08-07"),
    ("2022-10-01", "2022-10-07"),
    ("2022-12-01", "2022-12-07"),
    ("2023-04-01", "2023-04-07"),
    ("2023-06-01", "2023-06-07"),
    ("2023-07-01", "2023-07-07"),
    ("2023-08-01", "2023-08-07"),
    ("2023-10-01", "2023-10-07"),
    ("2023-12-01", "2023-12-07"),
    ("2024-04-01", "2024-04-07"),
    ("2024-06-01", "2024-06-07"),
    ("2024-07-01", "2024-07-07"),
    ("2024-08-01", "2024-08-07"),
    ("2024-10-01", "2024-10-07"),
    ("2024-12-01", "2024-12-07"),
    ("2025-04-01", "2025-04-07"),
    ("2025-06-01", "2025-06-07"),
    ("2025-07-01", "2025-07-07"),
    ("2025-08-01", "2025-08-07"),
    ("2025-10-01", "2025-10-07"),
    ("2025-12-01", "2025-12-07")
]


In [10]:
import json 
import requests 
from shapely.geometry import Point, box 
from shapely.ops import transform 
from pyproj import CRS, Transformer 
import numpy as np 
import matplotlib.pyplot as plt 

# Load the JSON file 
with open('wastewatertreatmentplants.json', 'r', encoding='utf-8') as f: 
    # wastewater_plants is a list of dictionaries, e.g., [{'name': 'ARTA', ...}, ...]
    wastewater_plants = json.load(f) 

# Define coordinate reference systems 
WGS84_CRS = CRS("EPSG:4326")        # Standard GPS coordinates (Degrees) 
GREEK_GRID_CRS = CRS("EPSG:2100")   # Greek Grid for accurate meters (Meters) 

# Create transformers 
transformer_to_greek_grid = Transformer.from_crs(WGS84_CRS, GREEK_GRID_CRS, always_xy=True).transform 
transformer_to_wgs84 = Transformer.from_crs(GREEK_GRID_CRS, WGS84_CRS, always_xy=True).transform 

# Function to create a 4 km square around a point 
def create_square_around_point(lon, lat, size_km=4): 
    # Convert the point to Greek Grid 
    x, y = transformer_to_greek_grid(lon, lat) 

    # Calculate the square's bounds in Greek Grid (meters) 
    half_size = size_km * 500  # 1 km = 1000 meters, half for each side 
    square = box(x - half_size, y - half_size, x + half_size, y + half_size) 

    # Convert the square back to WGS84 
    square_wgs84 = transform(transformer_to_wgs84, square) 

    return square_wgs84 

# Process each plant 
plant_squares = [] 
# 1. FIX: Iterate directly over the list
for plant in wastewater_plants: 
    # 2. FIX: Access coordinates directly from the object
    lon = plant['longitude'] 
    lat = plant['latitude'] 
    square = create_square_around_point(lon, lat) 
    plant_squares.append((plant['name'], square))

# Step 2: Load and Process Wastewater Treatment Plants
Add the following code to your notebook to load the JSON file and process each plant:

You need to check the structure of your JSON file and adjust the code accordingly. Here's how you can handle both cases:
1. Check the Structure of Your JSON File
Open the JSON file and check if it is structured as:

A list of features (e.g., [{...}, {...}, ...]), or
A GeoJSON FeatureCollection (e.g., {"type": "FeatureCollection", "features": [...]}). 

[{"code":"EL211001011","name":"ΑΡΤΑ","nameEn":"ARTA","latitude":39.135494,"longitude":20.991009,"year":1991,"priorityId":1,"priority":"Προτεραιότητα Α","priorityEn":"Priority A","compliance":true,"sludgeTreatmentMethod":"31s","wasteTreatmentMethod":"3NPm","capacity":36670,"administrativeRegionId":4,"administrativeRegion":"ΠΕΡΙΦΕΡΕΙΑ ΗΠΕΙΡΟΥ","administrativeRegionEn":"ΠΕΡΙΦΕΡΕΙΑ ΗΠΕΙΡΟΥ","municipalId":80,"municipal":"ΔΗΜΟΣ ΑΡΤΑΙΩΝ","municipalEn":"ΔΗΜΟΣ ΑΡΤΑΙΩΝ","reuse":false,"reuseMethods":null,"riverBasinId":22,"riverBasin":"Αράχθου","riverBasinEn":"Arachthou","riverBasinDistrictId":10,"riverBasinDistrict":"ΗΠΕΙΡΟΣ","riverBasinDistrictEn":"EPIRUS","receiverCode":"EL2110010110","receiverName":"ΑΡΑΧΘΟΣ Π.","receiverNameEn":"ARAHTHOS RIVER","rec

In [ ]:
import json
from shapely.geometry import Point, box
from shapely.ops import transform
from pyproj import CRS, Transformer

# Load the JSON file
with open('wastewatertreatmentplants.json', 'r', encoding='utf-8') as f:
    wastewater_plants = json.load(f)

# Define coordinate reference systems
WGS84_CRS = CRS("EPSG:4326")        # Standard GPS coordinates (Degrees)
GREEK_GRID_CRS = CRS("EPSG:2100")   # Greek Grid for accurate meters (Meters)

# Create transformers
transformer_to_greek_grid = Transformer.from_crs(WGS84_CRS, GREEK_GRID_CRS, always_xy=True).transform
transformer_to_wgs84 = Transformer.from_crs(GREEK_GRID_CRS, WGS84_CRS, always_xy=True).transform

# Function to create a 4 km square around a point
def create_square_around_point(lon, lat, size_km=4):
    # Convert the point to Greek Grid
    x, y = transformer_to_greek_grid(lon, lat)

    # Calculate the square's bounds in Greek Grid (meters)
    half_size = size_km * 500  # 1 km = 1000 meters, half for each side
    square = box(x - half_size, y - half_size, x + half_size, y + half_size)

    # Convert the square back to WGS84
    square_wgs84 = transform(transformer_to_wgs84, square)

    return square_wgs84

# Process each plant
plant_squares = []
for plant in wastewater_plants:
    # Extract latitude and longitude
    lon = plant['longitude']
    lat = plant['latitude']
    name = plant['name']

    # Create a 4 km square around the plant
    square = create_square_around_point(lon, lat)
    plant_squares.append((name, square))


### Step 3: Define Time Slots
Define the time slots for each year and month:

In [ ]:

time_slots = [
    ("2022-04-01", "2022-04-07"),
    ("2022-06-01", "2022-06-07"),
    ("2022-07-01", "2022-07-07"),
    ("2022-08-01", "2022-08-07"),
    ("2022-10-01", "2022-10-07"),
    ("2022-12-01", "2022-12-07"),
    ("2023-04-01", "2023-04-07"),
    ("2023-06-01", "2023-06-07"),
    ("2023-07-01", "2023-07-07"),
    ("2023-08-01", "2023-08-07"),
    ("2023-10-01", "2023-10-07"),
    ("2023-12-01", "2023-12-07"),
    ("2024-04-01", "2024-04-07"),
    ("2024-06-01", "2024-06-07"),
    ("2024-07-01", "2024-07-07"),
    ("2024-08-01", "2024-08-07"),
    ("2024-10-01", "2024-10-07"),
    ("2024-12-01", "2024-12-07"),
    ("2025-04-01", "2025-04-07"),
    ("2025-06-01", "2025-06-07"),
    ("2025-07-01", "2025-07-07"),
    ("2025-08-01", "2025-08-07"),
    ("2025-10-01", "2025-10-07"),
    ("2025-12-01", "2025-12-07")
]

print ("We read : ",len(plant_squares), " wastewater plants in Hellas Member State")
print ("We read : ",len(time_slots), " days/snapshots for every wastewater plant")
print ("We will process in total : ",len(time_slots) * len(plant_squares) , " unique satellite Sentinel 2 Images with 5Bands  per Image Total Number of raw files: ", 5*len(time_slots) * len(plant_squares))#B01 B02, B03, B04, B08


## New function REQUEST  Loop Through Plants and Time Slots
Update your get_request function to loop through both plants and time slots:

In [ ]:
import os
from datetime import datetime

# 1. Create the timestamped directory name
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_DIR = f"WASTEWATER_INVESTIGATION_{TIMESTAMP}"
print(f"Saving files to directory: {OUTPUT_DIR}")

# 2. Define the function with the necessary modifications
def get_request(plant_name, square, slot):
    # Ensure the output directory exists
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Extract the bounding box from the square
    minx, miny, maxx, maxy = square.bounds
    bbox = [minx, miny, maxx, maxy]

    # Custom filename for the TIFF file inside the tar
    custom_filename = f"{plant_name}_{slot[0]}.tif"

    request = {
        "input": {
            "bounds": {
                "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
                "bbox": bbox,
            },
            "data": [
                {
                    "dataFilter": {
                        "timeRange": {
                            "from": slot[0] + "T00:00:00Z",
                            "to": slot[1] + "T00:00:00Z",
                        }
                    },
                    "type": "sentinel-2-l2a",
                }
            ],
        },
        "output": {
            "width": 512,  # Adjust based on the resolution you need
            "height": 512,
            "responses": [
                {
                    "identifier": custom_filename,  # Custom filename for the TIFF,
                    "format": {"type": "image/tiff"},
                }
            ],
        },
        "evalscript": evalscript,
    }

    url = "https://sh.dataspace.copernicus.eu/api/v1/process"
    response = oauth.post(url, json=request, headers={"Accept": "application/tar"})
    print(f"Response for {plant_name} ({slot[0]}): {response.status_code}")

    if response.status_code in (200,):
        with open(f"tarfile_{plant_name}_{slot[0]}.tar", "wb") as tarfile:
            tarfile.write(response.content)

    return response


### Step 4: Download Sentinel-2 Images
Use the Sentinel Hub Process API to download images for each plant and time slot:

In [ ]:
from sentinelhub import SHConfig, BBox, CRS, MimeType, SentinelHubRequest
import matplotlib.pyplot as plt
import numpy as np
import tarfile

# Configure Sentinel Hub
#config = SHConfig()
#config.sh_client_id = ""  # Replace with your client ID
#config.sh_client_secret = ""  # Replace with your client secret
#config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
#config.sh_base_url = "https://sh.dataspace.copernicus.eu"

# Evalscript for SE2WQA
evalscript_se2waq = evalscript


# Execute the request

oauth = getauth_token()
responses = []

# Loop through each wastewater plant (plant_name, square)
# The square object contains the bounding box geometry
for plant_name, square in plant_squares:
    # Loop through each time slot (start_date, end_date)
    for slot in time_slots:
        response = get_request(plant_name, square, slot)
        responses.append(response)

print(f"Completed {len(responses)} requests for {len(plant_squares)} plants over {len(time_slots)} time time_slots.")


Step 5: Save Images
To save the images to your local machine, run the following code 

In [ ]:
import os
import zipfile
from IPython.display import FileLink

# Define the folder to zip and the output zip file name
folder_to_zip = "WASTEWATER_INVESTIGATION_20251004_142626"
output_zip_name = "WASTEWATER_INVESTIGATION_20251004_142626.zip"

# Create a zip file of the folder
with zipfile.ZipFile(output_zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(folder_to_zip):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, start=folder_to_zip)
            zipf.write(file_path, arcname)

print(f"Successfully created {output_zip_name}")

# Generate a download link for the zip file
FileLink(output_zip_name)

